In [1]:
#!/usr/bin/env python3
"""
Test script for understanding the Individual class
This represents a diploid organism (has 2 copies of each chromosome)
"""

import numpy as np
import random
from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Marker:
    id: str
    physical_position: float
    genetic_position: float

@dataclass
class Chromosome:
    id: int
    physical_length_bp: float
    genetic_length_cM: float
    markers: List[Marker]

class Individual:
    """
    Represents a diploid individual - has TWO copies of each chromosome.
    
    Key concepts:
    - maternal_chroms: Chromosomes inherited from mother
    - paternal_chroms: Chromosomes inherited from father  
    - Each chromosome copy stores 'alleles' (0 or 2) at each marker
    - Allele values represent ancestral population (0 = Pop A, 2 = Pop B)
    """
    
    def __init__(self, id: str, chromosomes: List[Chromosome]):
        self.id = id
        self.chromosomes = chromosomes
        self.maternal_chroms = {}  # Will store {chrom_id: {'alleles': [...], 'positions': [...]}}
        self.paternal_chroms = {}  # Same structure
        
    def initialise_ancestral_chromosomes(self, ancestry: int):
        """
        Sets up pure ancestral individual.
        
        ancestry = 0: All alleles are 0 (pure Population A)
        ancestry = 2: All alleles are 2 (pure Population B)
        
        For a pure individual, BOTH chromosome copies have the same ancestry.
        """
        print(f"Initializing {self.id} with pure ancestry {ancestry}")
        
        for chrom in self.chromosomes:
            n_markers = len(chrom.markers)
            
            # Both maternal and paternal chromosomes get the same ancestry
            maternal_alleles = [ancestry] * n_markers
            paternal_alleles = [ancestry] * n_markers
            
            # Store the alleles and positions
            self.maternal_chroms[chrom.id] = {
                'alleles': maternal_alleles,
                'positions': [m.physical_position for m in chrom.markers]
            }
            self.paternal_chroms[chrom.id] = {
                'alleles': paternal_alleles,
                'positions': [m.physical_position for m in chrom.markers]
            }
            
            print(f"  Chr {chrom.id}: {n_markers} markers, all set to ancestry {ancestry}")

def create_test_individual():
    """Create a test individual with sample chromosomes"""
    print("=== CREATING TEST INDIVIDUAL ===\n")
    
    # Create test chromosomes
    chromosomes = []
    for chrom_id in [1, 2]:
        markers = []
        for marker_idx in range(3):  # 3 markers per chromosome
            marker = Marker(
                id=f"chr{chrom_id}_marker_{marker_idx+1}",
                physical_position=(marker_idx + 1) * 1_000_000,  # 1M, 2M, 3M
                genetic_position=(marker_idx + 1) * 10.0         # 10, 20, 30 cM
            )
            markers.append(marker)
        
        chromosome = Chromosome(
            id=chrom_id,
            physical_length_bp=10_000_000,
            genetic_length_cM=100.0,
            markers=markers
        )
        chromosomes.append(chromosome)
    
    print(f"Created {len(chromosomes)} chromosomes:")
    for chrom in chromosomes:
        print(f"  Chr {chrom.id}: {len(chrom.markers)} markers")
    
    return chromosomes

def test_pure_individuals():
    """Test creating pure ancestral individuals"""
    print("\n=== TESTING PURE INDIVIDUALS ===\n")
    
    chromosomes = create_test_individual()
    
    # Create pure Population A individual
    print("1. Creating pure Population A individual...")
    individual_A = Individual("Pure_A", chromosomes)
    individual_A.initialise_ancestral_chromosomes(ancestry=0)
    
    # Create pure Population B individual  
    print("\n2. Creating pure Population B individual...")
    individual_B = Individual("Pure_B", chromosomes)
    individual_B.initialise_ancestral_chromosomes(ancestry=2)
    
    return individual_A, individual_B

def examine_individual_chromosomes(individual):
    """Look inside an individual's chromosomes"""
    print(f"\n=== EXAMINING {individual.id} ===")
    
    for chrom_id in individual.maternal_chroms:
        print(f"\nChromosome {chrom_id}:")
        
        mat_alleles = individual.maternal_chroms[chrom_id]['alleles']
        pat_alleles = individual.paternal_chroms[chrom_id]['alleles']
        positions = individual.maternal_chroms[chrom_id]['positions']
        
        print("  Marker    Position     Maternal  Paternal")
        print("  ------    --------     --------  --------")
        
        for i, pos in enumerate(positions):
            marker_name = f"M{i+1}"
            print(f"  {marker_name:<8}  {pos:>8,.0f}     {mat_alleles[i]:>8}  {pat_alleles[i]:>8}")

def test_hybrid_individual():
    """Test creating a hybrid individual manually"""
    print("\n=== TESTING HYBRID INDIVIDUAL ===\n")
    
    chromosomes = create_test_individual()
    
    # Create a hybrid individual manually
    hybrid = Individual("Manual_Hybrid", chromosomes)
    
    print("Creating hybrid individual manually...")
    for chrom in chromosomes:
        n_markers = len(chrom.markers)
        
        # Maternal chromosome: mix of ancestries
        maternal_alleles = [0, 2, 0]  # Pop A, Pop B, Pop A
        # Paternal chromosome: different mix
        paternal_alleles = [2, 2, 0]  # Pop B, Pop B, Pop A
        
        hybrid.maternal_chroms[chrom.id] = {
            'alleles': maternal_alleles,
            'positions': [m.physical_position for m in chrom.markers]
        }
        hybrid.paternal_chroms[chrom.id] = {
            'alleles': paternal_alleles,
            'positions': [m.physical_position for m in chrom.markers]
        }
    
    return hybrid

def calculate_test_metrics(individual):
    """Calculate hybrid index and heterozygosity for testing"""
    print(f"\n=== CALCULATING METRICS FOR {individual.id} ===")
    
    total_alleles = 0
    pop_0_alleles = 0
    heterozygous_markers = 0
    total_markers = 0
    
    for chrom_id in individual.maternal_chroms:
        mat_alleles = individual.maternal_chroms[chrom_id]['alleles']
        pat_alleles = individual.paternal_chroms[chrom_id]['alleles']
        
        for i in range(len(mat_alleles)):
            # Count total alleles and Pop 0 alleles
            total_alleles += 2
            if mat_alleles[i] == 0:
                pop_0_alleles += 1
            if pat_alleles[i] == 0:
                pop_0_alleles += 1
            
            # Count heterozygous markers
            total_markers += 1
            if mat_alleles[i] != pat_alleles[i]:
                heterozygous_markers += 1
    
    hybrid_index = pop_0_alleles / total_alleles if total_alleles > 0 else 0.0
    heterozygosity = heterozygous_markers / total_markers if total_markers > 0 else 0.0
    
    print(f"Total alleles: {total_alleles}")
    print(f"Pop 0 alleles: {pop_0_alleles}")
    print(f"Hybrid Index (proportion from Pop 0): {hybrid_index:.3f}")
    print(f"Heterozygous markers: {heterozygous_markers}/{total_markers}")
    print(f"Heterozygosity: {heterozygosity:.3f}")
    
    return hybrid_index, heterozygosity

if __name__ == "__main__":
    # Set seed for reproducibility
    random.seed(42)
    np.random.seed(42)
    
    # Test pure individuals
    pure_A, pure_B = test_pure_individuals()
    
    # Examine their chromosomes
    examine_individual_chromosomes(pure_A)
    examine_individual_chromosomes(pure_B)
    
    # Test hybrid individual
    hybrid = test_hybrid_individual()
    examine_individual_chromosomes(hybrid)
    
    # Calculate metrics
    print("\n" + "="*60)
    calculate_test_metrics(pure_A)
    calculate_test_metrics(pure_B)
    calculate_test_metrics(hybrid)
    
    print("\n=== SUMMARY ===")
    print("✓ Individuals have maternal and paternal chromosome copies")
    print("✓ Pure individuals have all alleles from one population (0 or 2)")
    print("✓ Hybrid index = proportion of alleles from Population 0")
    print("✓ Heterozygosity = proportion of markers with different maternal/paternal alleles")
    print("✓ Pure Pop A: hybrid index = 1.0, heterozygosity = 0.0")
    print("✓ Pure Pop B: hybrid index = 0.0, heterozygosity = 0.0")
    print("✓ Hybrids: intermediate hybrid index, variable heterozygosity")


=== TESTING PURE INDIVIDUALS ===

=== CREATING TEST INDIVIDUAL ===

Created 2 chromosomes:
  Chr 1: 3 markers
  Chr 2: 3 markers
1. Creating pure Population A individual...
Initializing Pure_A with pure ancestry 0
  Chr 1: 3 markers, all set to ancestry 0
  Chr 2: 3 markers, all set to ancestry 0

2. Creating pure Population B individual...
Initializing Pure_B with pure ancestry 2
  Chr 1: 3 markers, all set to ancestry 2
  Chr 2: 3 markers, all set to ancestry 2

=== EXAMINING Pure_A ===

Chromosome 1:
  Marker    Position     Maternal  Paternal
  ------    --------     --------  --------
  M1        1,000,000            0         0
  M2        2,000,000            0         0
  M3        3,000,000            0         0

Chromosome 2:
  Marker    Position     Maternal  Paternal
  ------    --------     --------  --------
  M1        1,000,000            0         0
  M2        2,000,000            0         0
  M3        3,000,000            0         0

=== EXAMINING Pure_B ===

Ch